In [13]:
import nltk
import re
import unicodedata
import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [14]:
# Replace the file path with the correct one
file_path = r"C:\Users\91702\Downloads\train.csv"

# Try reading the file with 'latin1' encoding

df = pd.read_csv(file_path, encoding='latin1')
# Further processing or modifications as needed
df.drop(df[df['sentiment'] == "neutral"].index, inplace=True)
print(df)

           textID                                               text  \
1      549e992a42      Sooo SAD I will miss you here in San Diego!!!   
2      088c60f138                          my boss is bullying me...   
3      9642c003ef                     what interview! leave me alone   
4      358bd9e861   Sons of ****, why couldn`t they put them on t...   
6      6e0c6d75b1  2am feedings for the baby are fun when he is a...   
...           ...                                                ...   
27475  b78ec00df5                                     enjoy ur night   
27476  4eac33d1c0   wish we could come see u on Denver  husband l...   
27477  4f4c4fc327   I`ve wondered about rake to.  The client has ...   
27478  f67aae2310   Yay good for both of you. Enjoy the break - y...   
27479  ed167662a5                         But it was worth it  ****.   

                    selected_text sentiment Time of Tweet Age of User  \
1                        Sooo SAD  negative          noon     

In [15]:
# Replace the file path with the correct one
file_path1 = r"C:\Users\91702\Downloads\test.csv"

# Try reading the file with 'latin1' encoding

df1 = pd.read_csv(file_path1, encoding='latin1')
# Further processing or modifications as needed
df1.drop(df1[df1['sentiment'] == "neutral"].index, inplace=True)
df1.dropna(inplace=True)
print(df1)

          textID                                               text sentiment  \
1     96d74cb729   Shanghai is also really exciting (precisely -...  positive   
2     eee518ae67  Recession hit Veronique Branquinho, she has to...  negative   
3     01082688c6                                        happy bday!  positive   
4     33987a8ee5             http://twitpic.com/4w75p - I like it!!  positive   
5     726e501993                    that`s great!! weee!! visitors!  positive   
...          ...                                                ...       ...   
3529  e5f0e6ef4b  its at 3 am, im very tired but i can`t sleep  ...  negative   
3530  416863ce47  All alone in this old house again.  Thanks for...  positive   
3531  6332da480c   I know what you mean. My little dog is sinkin...  negative   
3532  df1baec676  _sutra what is your next youtube video gonna b...  positive   
3533  469e15c5a8   http://twitpic.com/4woj2 - omgssh  ang cute n...  positive   

     Time of Tweet Age of U

In [16]:
nltk.download('stopwords')

# Instantiate tokenizer, stemmer, and stopwords
tokenizer = RegexpTokenizer(r'\w+')
ps = PorterStemmer()
en_stopwords = set(stopwords.words('english'))

# Function for cleaning text
from nltk.stem import WordNetLemmatizer

# Initialize the WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Function for cleaning text with Lemmatization
def getCleanedText(text):
    # Convert to string (in case it's not already)
    #text = str(text)
    
    # Remove digits
    clean_text = re.sub(r'\d+', '', text)
    
    # Remove URLs
    clean_text = re.sub(r'http\S+|www.\S+|mailto:\S+', '', clean_text)
    
    clean_text = re.sub(r'#\w+', '', text)

    # Remove mentions
    clean_text = re.sub(r'@\w+', '', text)
    
    # Remove non-alphabetic characters
    clean_text = re.sub(r'[^a-zA-Z\s]', '', clean_text)
    
    # Remove extra spaces
    clean_text = ' '.join(clean_text.split())
    
    # Remove diacritics (accents)
    clean_text = ''.join(c for c in unicodedata.normalize('NFD', clean_text) if unicodedata.category(c) != 'Mn')
    
    # Convert to lowercase
    clean_text = clean_text.lower()
    
    # Tokenize the cleaned text
    tokens = tokenizer.tokenize(clean_text)
    
    # Remove stopwords
    new_tokens = [token for token in tokens if token not in en_stopwords]
    
    # Lemmatization
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in new_tokens]
    
    # Join tokens back into a cleaned text
    clean_text = " ".join(lemmatized_tokens)
    
    return clean_text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\91702\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
# Creating a new column of cleaned data
df['new'] = df['text'].apply(getCleanedText)
df1['new'] = df1['text'].apply(getCleanedText)

In [20]:
# Assigning the values for x_train, y_train, x_test, y_test
x_train = df['new']
y_train = df['sentiment']
x_test = df1['new']
y_test = df1['sentiment']

In [21]:
# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=25000)  # You can adjust max_features as needed
x_train_tfidf = tfidf_vectorizer.fit_transform(x_train)
x_test_tfidf = tfidf_vectorizer.transform(x_test)

# Initialize SVC classifier
classifier = SVC(kernel = 'rbf', random_state = 0)

# Train the classifier
classifier.fit(x_train_tfidf, y_train)

# Make predictions on the test set
y_pred = classifier.predict(x_test_tfidf)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 89.02%
